# Przygotowanie danych - Mikołaj Pacek, Karol Mierzwiński

In [44]:

from sympy import Integer
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import requests
import hashlib
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
import plotly.express as px

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split


Basic import and usage

In [45]:
#url = 'https://raw.githubusercontent.com/wiktorDeka/WykrywanieAnomalii/main/reduced_dataset_sdn.csv'
url = 'https://raw.githubusercontent.com/wiktorDeka/WykrywanieAnomalii/main/dataset_sdn.csv'
s = requests.get(url).content

counters = pd.read_csv(io.StringIO(s.decode('utf-8')))
counters.dt = pd.to_datetime(counters.dt, unit='s')
counters.set_index('dt', inplace=True)
counters.fillna(value=0, inplace=True)

#ip embedding
#target encoding
#bucketing
#hash semantyczny

#isolation forest / SVM
#pca
# Specify the percentage of data you want to use for training

## Visualization of UDP Data

In [18]:
udp_data = counters[counters['Protocol'] == 'UDP']
udp_data['log_bytecount'] = np.log(udp_data['bytecount'] + 1)

# 1. Source IP Count
src_ip_pkt_count = udp_data.groupby(['dt', 'src'])['pktcount'].sum().reset_index()
top_src_ips = src_ip_pkt_count['src'].value_counts().index[:5]
for ip in top_src_ips:
    ip_data = src_ip_pkt_count[src_ip_pkt_count['src'] == ip]
    plt.scatter(ip_data['dt'], ip_data['pktcount'], label=ip)

plt.title('Packet Count from Top Source IPs Over Time')
plt.xlabel('Time')
plt.ylabel('Packet Count')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# 2. Destrination IP Count
dst_ip_pkt_count = udp_data.groupby(['dt', 'dst'])['pktcount'].sum().reset_index()

# Now, let's plot the top 5 destination IPs that received the most packets
top_dst_ips = dst_ip_pkt_count['dst'].value_counts().index[:5]

for ip in top_dst_ips:
    ip_data = dst_ip_pkt_count[dst_ip_pkt_count['dst'] == ip]
    plt.scatter(ip_data['dt'], ip_data['pktcount'], label=ip)

plt.title('Packet Count to Top Destination IPs Over Time')
plt.xlabel('Time')
plt.ylabel('Packet Count')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# 3. Average Packet Size
avg_pkt_size = udp_data.groupby('dt')['bytecount'].mean()
plt.scatter(avg_pkt_size.index, avg_pkt_size)
plt.title('Average Packet Size Over Time')
plt.xlabel('Time')
plt.ylabel('Average Packet Size')
plt.xticks(rotation=45)

plt.show()

# 4. Packet Rate
# pkt_rate = udp_data['pktcount'].div(udp_data['dt'].diff().dt.seconds).fillna(0)
pkt_rate = udp_data['pktrate']
plt.scatter(udp_data.index, pkt_rate)
plt.title('Packet Rate Over Time')
plt.xlabel('Time')
plt.ylabel('Packet Rate')
plt.xticks(rotation=45)

plt.show()



KeyError: 'src'

## Przygotowanie danych – brakujące dane

Usunięcie całych obserwacji w których występuje brak danych. Często metodę określa się z angielskiego jako complete-case analysis (CCA), czyli analizie poddajemy tylko te obserwacje dla których mamy pełne dane dla wszystkich zmiennych. W naszym przypadku nie posiadamy takich rekordów danych w którym brakowałoby jakichś parametrów.

In [ ]:
missing = [var for var in counters.columns if counters[var].isnull().sum() > 0]
missing

[]

## Przygotowanie danych - kodowanie zmiennych kategorialnych

Dla przypomnienia zmienna kategorialna to taka, która pozwala na przeprowadzenie klasyfikacji serii danych za pomocą wartości związanych z określoną jakością czy kategorią. Zmienna kategorialna pozwala sklasyfikować, pogrupować w oparciu o informację jakościową, czyli najczęściej w danych spotkamy zmienną kategorialną z elementami stanowiącymi łańcuch tekstowy. W naszym przypadku będziemy zaliczać do zmiennych kategorialnych : data/czas danych wpisów, nazwa protokołu jaki został uzyty, destination i source adresy.

Na podstawie róznych benchamrków, mozemy się skupić na tych 4 enkoderach.

"We observed that four encoders: One-Hot, Binary (‘Bin’ on the plot), Sum, and Weight of Evidence are consistently among the best performing. For logistic regression, the difference of these four from the rest is statistically significant."


## Przygotowanie danych - duration

Łączymy duration_sec i duration_nsec, aby otrzymać pełny czas trwania - duration.


In [ ]:
# Create the new 'duration' column
counters['duration'] = counters['dur'] + counters['dur_nsec'] / 1e9

# Drop the 'duration_sec' and 'duration_nsec' columns
counters = counters.drop(['dur', 'dur_nsec'], axis=1)


## Normalizacja

Normalizacja to proces przekształcania danych wejściowych w taki sposób, aby poprawić wydajność modelu uczącego się. Głównym celem normalizacji jest doprowadzenie danych do jednolitej skali lub rozkładu, co może ułatwić uczenie modelu poprzez zmniejszenie wpływu różnic w zakresach lub jednostkach między różnymi cechami danych.

Istnieje kilka popularnych metod normalizacji danych, takich jak:

Normalizacja min-max: Skaluje wartości cech do zakresu między 0 a 1, wykorzystując minimum i maksimum wartości danej cechy.
Z-score normalization (StandardScaler): Przekształca wartości cech tak, aby miały średnią równą zero i odchylenie standardowe równe jeden.
Normalizacja jednostkowa: Polega na dzieleniu każdej wartości cechy przez normę wektora cechy, co prowadzi do przeskalowania cech do jednostkowego wektora o długości 1.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Assuming 'counters' is your DataFrame
columns_to_normalize = ['pktcount', 'bytecount', 'tx_bytes', 'rx_bytes']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to your data and transform it
counters[columns_to_normalize] = scaler.fit_transform(counters[columns_to_normalize])
counters

,switch,src,dst,pktcount,bytecount,tot_dur,flows,packetins,pktperflow,byteperflow,...,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label,duration
dt,,,,,,,,,,,,,,,,,,,,,
1970-01-01 03:10:25,1,10.0.0.1,10.0.0.8,0.334095,0.334095,1.010000e+11,3,1943,13535,14428310,...,0,UDP,3,1.133293e-01,3.019397e-06,0,0,0,0,100.716
1970-01-01 03:13:25,1,10.0.0.1,10.0.0.8,0.935875,0.935875,2.810000e+11,2,1943,13531,14424046,...,0,UDP,4,7.850523e-07,2.618628e-06,0,0,0,0,280.734
1970-01-01 03:10:25,1,10.0.0.2,10.0.0.8,0.668257,0.668257,2.010000e+11,3,1943,13534,14427244,...,0,UDP,1,7.480438e-07,3.190001e-07,0,0,0,0,200.744
1970-01-01 03:10:25,1,10.0.0.2,10.0.0.8,0.668257,0.668257,2.010000e+11,3,1943,13534,14427244,...,0,UDP,2,6.637904e-07,5.713736e-07,0,0,0,0,200.744
1970-01-01 03:10:25,1,10.0.0.2,10.0.0.8,0.668257,0.668257,2.010000e+11,3,1943,13534,14427244,...,0,UDP,3,4.472514e-07,2.765004e-06,0,0,0,0,200.744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 03:11:25,4,10.0.0.2,10.0.0.8,0.866800,0.866800,2.590000e+11,6,1943,13533,14426178,...,0,UDP,2,1.018914e-06,1.263889e-01,0,3837,3837,0,258.834
1970-01-01 03:11:25,4,10.0.0.2,10.0.0.8,0.866800,0.866800,2.590000e+11,6,1943,13533,14426178,...,0,UDP,3,1.593722e-01,3.231390e-06,7676,0,7676,0,258.834
1970-01-01 03:11:25,4,10.0.0.13,10.0.0.8,0.486894,0.475885,2.110000e+11,6,1943,9287,9677054,...,0,UDP,3,1.141751e-06,2.403630e-01,0,10242,10242,1,210.860


## Hash encoding

Hash encoding to przydatne kodowanie w przypadku dużych zbiorów danych, ponieważ wymaga mniej pamięci niż kodowanie jednokrotne i jest szybsze niż inne techniki kodowania. Kodowanie skrótu odwzorowuje każdą kategorię na unikalną wartość całkowitą w predefiniowanym zakresie przy użyciu funkcji skrótu . Mogą jednak wystąpić kolizje prowadzące do zmniejszenia wydajności modelu. My w naszym przypadku, będziemy hashować adresy ip używając hasha SHA256.

In [ ]:

src_encoded = counters['src'].apply(lambda x: int(hashlib.sha256(str(x).encode('utf-8')).hexdigest(), 16))
counters['src'] = src_encoded


dst_encoded = counters['dst'].apply(lambda x: int(hashlib.sha256(str(x).encode('utf-8')).hexdigest(), 16))
counters['dst'] = dst_encoded

print(type(counters['src'][0]))

counters



<class 'int'>


,switch,src,dst,pktcount,bytecount,tot_dur,flows,packetins,pktperflow,byteperflow,...,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label,duration
dt,,,,,,,,,,,,,,,,,,,,,
1970-01-01 03:10:25,1,1108245108271806112863518848100079671513901967...,8773506938910239794491519534150457445184585175...,0.334095,0.334095,1.010000e+11,3,1943,13535,14428310,...,0,UDP,3,1.133293e-01,3.019397e-06,0,0,0,0,100.716
1970-01-01 03:13:25,1,1108245108271806112863518848100079671513901967...,8773506938910239794491519534150457445184585175...,0.935875,0.935875,2.810000e+11,2,1943,13531,14424046,...,0,UDP,4,7.850523e-07,2.618628e-06,0,0,0,0,280.734
1970-01-01 03:10:25,1,9198774319483998541402195860639125225801762867...,8773506938910239794491519534150457445184585175...,0.668257,0.668257,2.010000e+11,3,1943,13534,14427244,...,0,UDP,1,7.480438e-07,3.190001e-07,0,0,0,0,200.744
1970-01-01 03:10:25,1,9198774319483998541402195860639125225801762867...,8773506938910239794491519534150457445184585175...,0.668257,0.668257,2.010000e+11,3,1943,13534,14427244,...,0,UDP,2,6.637904e-07,5.713736e-07,0,0,0,0,200.744
1970-01-01 03:10:25,1,9198774319483998541402195860639125225801762867...,8773506938910239794491519534150457445184585175...,0.668257,0.668257,2.010000e+11,3,1943,13534,14427244,...,0,UDP,3,4.472514e-07,2.765004e-06,0,0,0,0,200.744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 03:11:25,4,9198774319483998541402195860639125225801762867...,8773506938910239794491519534150457445184585175...,0.866800,0.866800,2.590000e+11,6,1943,13533,14426178,...,0,UDP,2,1.018914e-06,1.263889e-01,0,3837,3837,0,258.834
1970-01-01 03:11:25,4,9198774319483998541402195860639125225801762867...,8773506938910239794491519534150457445184585175...,0.866800,0.866800,2.590000e+11,6,1943,13533,14426178,...,0,UDP,3,1.593722e-01,3.231390e-06,7676,0,7676,0,258.834
1970-01-01 03:11:25,4,7568132274558340019932695141431029454339662908...,8773506938910239794491519534150457445184585175...,0.486894,0.475885,2.110000e+11,6,1943,9287,9677054,...,0,UDP,3,1.141751e-06,2.403630e-01,0,10242,10242,1,210.860


## Kodowanie One-Hot Encoding/DummyVariable Encoding

Dla zmiennych kategorialnych bez uporządkowania kodowanie numeryczne, a więc określające relację między kategoriami może prowadzić do niewłaściwego działania uczonego modelu. Wymuszenie uporządkowania, tak jak zrobiliśmy to poprzednio też nie jest dobrym rozwiązaniem. Dla takich przypadków stosuje się kodowanie One-Hot Encoding, które w języku polskim określa się jako kodowanie „1 z n”. Poniewaz dane wypisane powyzej mają mieć tylko postać rozróznienia kategorii / grup osoób, skorzystamy z tego kodowanie. W naszym wypadku występuje jednak problem ilości róznych kategorii w odniesieniu do adresów IP

In [ ]:
# ! jak cos to to jest tescik bo nwm czm pokazuje mi ograniczony zakres

# Determine categorical variables in the dataset
categorical_variables = [var for var in counters.columns if counters[var].dtype == 'O']

# Let's explore the cardinality in variables (src, dst, Protocol)
categories = {}
for cat_variable in categorical_variables:
    categories[counters[cat_variable].name] = list(counters[cat_variable].unique())

usecols = ['Protocol']

df = counters[usecols]

df = df.fillna(value='missing')

encoded_columns = pd.get_dummies(df, drop_first=False, dtype=int)

counters = pd.concat([counters, encoded_columns], axis=1)

counters.drop(columns=usecols, inplace=True)

sns.displot(data=counters, x='src',y='dst',col='Protocol');



ValueError: Could not interpret value `protocol_UDP` for `y`. An entry with this name does not appear in `data`.